# Coding A Rank Order Assignment Algorithm

The goal of the project was to write a piece of software to matche N Doctors with K Hospitals with open residency positions. Each Doctor provides a ranked list of their preference for Hospitals, without Hospitals ranking the Doctors.  In our scenario, the Hospitals provided a set number of positions they wish to fill.

### Section 1: Background
The Hungarian algorithm is a method for solving the assignment problem, where the goal is to optimally match workers to tasks at minimum cost. It works by transforming a cost matrix through several steps. First it subtracts the smallest value in each row **and then** in each column so that every row and column has at least one zero. Next, it attempts to cover all zeros with the minimum number of horizontal and vertical lines; if the number of lines equals the size of the matrix (which must be square), an optimal assignment can be made. If not, the algorithm adjusts the uncovered entries by subtracting the smallest uncovered value from them and adding it at intersections of covering lines to obtain more options, then repeats the covering process. This iterative adjustment ensures that the solution converges to one where zeros represent the optimal assignment, allowing a one-to-one matching of workers to tasks.

For a more in depth look at the Hungarian Algorithm, you can reference <a href="https://www.topcoder.com/thrive/articles/Assignment%20Problem%20and%20Hungarian%20Algorithm" target="_parent"> this article on topcoder.com.

### Section 2: The Data

We assume our data has been collected into a spreadsheet (a CSV file) and organized so that each doctor has their own row, which is itself populated with preference rankings [1, K] for each hospital in it's corresponding column.  

We load this in as a **PANDAS** dataframe.

In [27]:
import Hungarian_Alg_Steps as HAS
import Hungarian_Alg_LineCheck as HAL
import HungarianAlgoImportData as HAID

import numpy as np
import pandas as pd

In [ ]:
df = HAID.import_data('Tests/Large_Test_500.csv')
display(df)

,Hospital,H1,H2,H3,H4,H5,H6,H7
0,Num,7,7,7,7,7,7,7
1,D1,3,3,7,1,1,7,1
2,D2,5,6,3,6,5,2,5
3,D3,5,7,3,7,7,6,7
4,D4,3,5,3,1,3,7,3
...,...,...,...,...,...,...,...,...
496,D496,6,3,5,4,4,5,4
497,D497,1,7,5,3,3,2,7
498,D498,2,4,1,4,1,1,1
499,D499,6,1,3,6,5,1,5


From this dataframe, we extract the details provided for each hospital and strip the dataframe of its row and column labels to allow us to work with just the cost matrix. 

In [25]:
hi = HAID.get_hospital_info(df)

prepped_df = HAID.prep_data(df)
prepped_array = prepped_df.to_numpy(copy = True)

print(f'get_hospital_info returns the number of positions available per hospital\n\n {hi}')
print(f'\nAnd we use this information to fill out the matrix, which then gets padded to become square\n\n {prepped_df}')
print(f'\nAt the end of the importing process we are left with the 2D array:\n\n {prepped_array}')

get_hospital_info returns the number of positions available per hospital

    H1  H2  H3  H4  H5  H6  H7
0   7   7   7   7   7   7   7

And we use this information to fill out the matrix, which then gets padded to become square

      H1_0  H2_0  H3_0  H4_0  H5_0  H6_0  H7_0  H1_1  H1_2  H1_3  ...  \
1       3     3     7     1     1     7     1     3     3     3  ...   
2       5     6     3     6     5     2     5     5     5     5  ...   
3       5     7     3     7     7     6     7     5     5     5  ...   
4       3     5     3     1     3     7     3     3     3     3  ...   
5       3     2     4     4     4     5     4     3     3     3  ...   
..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
496     6     3     5     4     4     5     4     6     6     6  ...   
497     1     7     5     3     3     2     7     1     1     1  ...   
498     2     4     1     4     1     1     1     2     2     2  ...   
499     6     1     3     6     5     1     5     

### Section 3: Reduce the Matrix

With our cost matrix created we then begin the steps of the Hungarian Algorithm.  

As previously explained, the first step is to reduce the matrix by subtracting the minimum value of each row from each row and repeating that for each column. 

In [5]:
row_reduced_matrix = HAS.step1_row_reduction(prepped_array)
print(f'Our original cost matrix was:\n {prepped_array}\n\n and following our row reduction we are given:\n {row_reduced_matrix}')

reduced_matrix = HAS.step2_col_reduction(row_reduced_matrix)
print(f'\nAfter column reduction, we obtain our final reduced matrix:\n {reduced_matrix}')



Our original cost matrix was:
 [[3 3 7 ... 0 0 0]
 [5 6 3 ... 0 0 0]
 [5 7 3 ... 0 0 0]
 ...
 [2 4 1 ... 0 0 0]
 [6 1 3 ... 0 0 0]
 [4 1 4 ... 0 0 0]]

 and following our row reduction we are given:
 [[3. 3. 7. ... 0. 0. 0.]
 [5. 6. 3. ... 0. 0. 0.]
 [5. 7. 3. ... 0. 0. 0.]
 ...
 [2. 4. 1. ... 0. 0. 0.]
 [6. 1. 3. ... 0. 0. 0.]
 [4. 1. 4. ... 0. 0. 0.]]

After column reduction, we obtain our final reduced matrix:
 [[2. 2. 6. ... 0. 0. 0.]
 [4. 5. 2. ... 0. 0. 0.]
 [4. 6. 2. ... 0. 0. 0.]
 ...
 [1. 3. 0. ... 0. 0. 0.]
 [5. 0. 2. ... 0. 0. 0.]
 [3. 0. 3. ... 0. 0. 0.]]


### Section 4: Crossing Out the Zeros

With the matrix both row and column reduced, we proceed to the process of "crossing out zeros" with the minimal number of lines.

Unfortunately, we are unaware of a python function that draws lines in a numpy array.  So, we turn to matrix manipulations.



*To begin, we pass the numpy array through and convert it into a boolean array where values indicate whether the entry is 0 (True) or not (False).
A copy of this array is passed to a helper that fills in a first set of zeros simply by picking the first occurence in each row.
The coordinates that we get back are then separated by row into row and column.*

*We then sift through the boolean matrix column by column in order to iteratively obtain the minimal horizontal and vertical "lines" needed.*


In [6]:
row_lines, col_lines, marked_zeros = HAL.Step_3_Line_Check(reduced_matrix)

print(f'This step returns the rows and columns that have lines crossing through them:')
print(f'Rows: {row_lines}')
print(f'Columns: {col_lines}')
print(f'Marked Zeros:\n {marked_zeros}')
print('\nAnd by finding the length of the line arrays and summing them, we can determine whether we move on to step 4 or 5')
print(f'Row lines({len(row_lines)}) + Column lines({len(col_lines)}) = {len(row_lines) + len(col_lines)}.')

if len(row_lines) + len(col_lines) == len(reduced_matrix):
    print("\nWe're moving to Step 5!!")
    fork = 5
else:
    print("\nWe have to do Step 4!")
    fork = 4

This step returns the rows and columns that have lines crossing through them:
Rows: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225

### Section 5: Adjusting the Matrix

If we fail to transform the matrix such that an optimal assignment can be obtained, we need to move to uncover new zeros.

The process works by identifying uncovered cells (those not included in any row or column covered by a line) and modifying their values relative to the smallest uncovered entry. The goal is to create new zeros in the uncovered region and preserve the zeros we already have. Let’s go through this process step by step.

We **first** determine entries that are *not* covered by any of the drawn lines.  Then we select the smallest of these values, called here m.

"m" is then *subtracted from all uncovered entries* and *added to intersection entries* (intersection entries are entries at the intersection of a row and column line from Step 3).

STEP4 then returns the updated cost matrix which gets passed back to STEP3 until we have N lines.

In [7]:
if fork == 4:
    while len(row_lines) + len(col_lines) != len(reduced_matrix):
        print(f'First we have {len(row_lines) + len(col_lines), len(reduced_matrix)}')
        reduced_matrix = HAS.step4_adjust_matrix(reduced_matrix, row_lines, col_lines)
        row_lines, col_lines, marked_zeros = HAL.Step_3_Line_Check(reduced_matrix)

print(f'Then we move on to Step5 because we have {len(row_lines) + len(col_lines), len(reduced_matrix)}')

Then we move on to Step5 because we have (500, 500)


### Section 6: Finding the Optimal Assignments

Once we have reduced the cost matrix and identified the zeros, the next step involves assigning rows to columns. This process, implemented in the function STEP5_find_solution, seeks to assign each row to a unique column while ensuring only 1:1 matching and, hopefully, efficiency.

To ensure we get our optimal assignments efficiently, the function does 2 passes.  The first pass goes ahead with directly matching any doctor that only has 1 candidate for assignment (i.e. we fully process rows with only 0).

The second (and beyond) passes take care of matching candidates with the next lowest number of options.  Repeating this until we have no more unmatched candidates (though if the matrix was padded, they may, in reality, be unmatched).

This gives us our final result, which is evaluated relative to the worst case scenario $ \left( 1 - \frac{\text{score}}{\text{max\_score}} \right) \times 100 \% $

Note we do not incorporate a penalty for unmatched individuals, because that **only occurs when there were not enough positions to begin with**.


In [23]:
assignments = HAL.STEP5_find_solution(marked_zeros)
num_to_match = len(HAID.get_doctors(df))
matches_df = HAL.match_residents(assignments, df)

# Dropping sorting by hospital and dropping the row indices for style
matches_df = matches_df.sort_values("Hospital Position")
matches_df = matches_df.reset_index(drop=True)

matches_numeric = HAL.match_residents_numeric(assignments, prepped_df, num_to_match)
score, max_score = HAID.get_score(prepped_df, matches_numeric)
display(matches_df)
print(f'\nWe have achieved a solution of: {(1 - score / max_score) * 100:.1f}%')

,Doctor,Hospital Position
0,D8,H1_0
1,D14,H1_1
2,D22,H1_2
3,D26,H1_3
4,D41,H1_4
...,...,...
495,D116,No Match_95
496,D117,No Match_96
497,D119,No Match_97
498,D122,No Match_98



We have achieved a solution of: 98.5%
